# Pre-processing the Data

This script will be used to prepare and featurize the train and test splits from the Direction of Voice Dataset from the FIGLAB.

Training Data:
- Room setup 1
- All utterance types
- All speakers except speakers 2 and 9

Test Data:
- Room setup 2
- All utterance types
- Only speakers 2 and 9

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display

from IPython import display
from matplotlib import pyplot

In [2]:
class Audio:
    '''
    A simple wrapper class for (1-channel) audio data
    data is a 1-D NumPy array containing the data
    rate is a number expressing the samples per second
    '''
    
    def __init__(self, data, rate):
        self.data = data
        self.rate = rate
        
    def play(self):
        return display.Audio(self.data, rate=self.rate)
    
    def plot_wave(self):
        librosa.display.waveplot(self.data, sr=self.rate)
        
    def plot_spectrum(self):
        n_fft = int(self.rate / 20)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(self.data, n_fft)), ref=np.max)
        librosa.display.specshow(D, y_axis='linear', sr=self.rate, hop_length=n_fft/4)
        
    @classmethod
    def fromfile(cls, fn):
        return cls(*librosa.load(fn, sr=None))

In [3]:
# Credit to Yihui Xiong, 2017, 
# https://github.com/xiongyihui/tdoa/blob/master/gcc_phat.py
def gcc_phat(sig, refsig, fs=1, max_tau=None, interp=16):
    '''
    This function computes the offset between the signal sig and the reference signal refsig
    using the Generalized Cross Correlation - Phase Transform (GCC-PHAT) method.
    '''
    
    # make sure the length for the FFT is larger or equal than len(sig) + len(refsig)
    n = sig.shape[0] + refsig.shape[0]

    # Generalized Cross Correlation Phase Transform
    SIG = np.fft.rfft(sig, n=n)
    REFSIG = np.fft.rfft(refsig, n=n)
    R = SIG * np.conj(REFSIG)

    cc = np.fft.irfft(R / np.abs(R), n=(interp * n))

    max_shift = int(interp * n / 2)
    if max_tau:
        max_shift = np.minimum(int(interp * fs * max_tau), max_shift)

    cc = np.concatenate((cc[-max_shift:], cc[:max_shift+1]))

    # find max cross correlation index
    shift = np.argmax(np.abs(cc)) - max_shift

    tau = shift / float(interp * fs)
    
    return tau, cc

In [4]:
def get_data_split(subject_nums, room):
    '''
    This function specifies that data split by subject number and trial number
    and returns the corresponding recording paths and relative recording paths
    for the data split
    '''
    data_dir = os.path.join(os.path.abspath('.'), 'data/raw/')
    subject_names = ['s' + str(i) for i in subject_nums]
    trial_names = [f'{s}_trial{t}'
                   for t in range(1, 2+1)
                   for s in [f'{room}_nowall', f'{room}_wall']]
    angle_names = [pos[0] + angle[0] + '_' + pos[1] + '_' + angle[1]
                   for pos in [('A', '1'), ('B', '3'), ('C', '5')]
                   for angle in [(str(i), str(45*i)) for i in range(3)]]
    trial_paths = [os.path.join(s, s + '_' + t)
                   for s in subject_names
                   for t in trial_names]
    recording_paths = [os.path.join(data_dir, t, a) 
                       for t in trial_paths
                       for a in angle_names]
    rel_recording_paths = [(s, t, a) for s in subject_names
                                     for t in trial_names
                                     for a in angle_names]
    return(recording_paths, rel_recording_paths)

In [5]:
# Training
train_subjects = ['1', '3', '4', '5', '6', '7', '8', '10']
train_recording_paths, train_rel_recording_paths = get_data_split(train_subjects, 'downstairs')

# Testing
test_subjects = ['2', '9']
test_recording_paths, test_rel_recording_paths = get_data_split(test_subjects, 'upstairs')

In [6]:
col_names = ['sample', 'trial', 'angle_pos', 'dov',
             *[f'gccphat_{i}_{j}_{d}' for i in range(4) for j in range(i+1, 4) for d in ['maxshift', 'auc', 'peakval']],
             *[f'gccphatval_{i}_{j}_{k}' for i in range(4) for j in range(i+1, 4) for k in range(23)]]

In [7]:
def get_featurized_data(recording_paths, rel_recording_paths):
    df = pd.DataFrame(columns=col_names)
    for pth_ind in range(len(recording_paths)):
        for angle in map(lambda i:45*i, range(360//45)):
            channel_recordings = [os.path.join(recording_paths[pth_ind], f'recording{r}_{angle}_{i}.wav') for r in range(2) for i in range(1,5)]
            print(channel_recordings)
            audio_files = [Audio.fromfile(r) for r in channel_recordings]
            four_channels = [a.data for a in audio_files]
            data_row = {
                'sample': rel_recording_paths[pth_ind][0],
                'trial': rel_recording_paths[pth_ind][1],
                'angle_pos': rel_recording_paths[pth_ind][2],
                'dov': angle 
            }        
            for i in range(4):
                for j in range(i+1,4):
                    gcc_phat_data = gcc_phat(four_channels[i], four_channels[j], 
                                          fs = audio_files[0].rate, max_tau=0.236 * 1e-3, interp=1)
                    data_row[f'gccphat_{i}_{j}_peakval'] = gcc_phat_data[1][11]
                    data_row[f'gccphat_{i}_{j}_auc'] = np.sum(gcc_phat_data[1])
                    data_row[f'gccphat_{i}_{j}_maxshift'] = gcc_phat_data[0]
                    for k in range(23):
                        data_row[f'gccphatval_{i}_{j}_{k}'] = gcc_phat_data[1][k]
            df = df.append(data_row, ignore_index=True)
    return df    

In [8]:
train_df = get_featurized_data(train_recording_paths, train_rel_recording_paths)
test_df = get_featurized_data(test_recording_paths, test_rel_recording_paths)

['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording0_0_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording0_0_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording0_0_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording0_0_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording1_0_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording1_0_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording1_0_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording1_0_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A0_1_0/recording0_45_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_tr

['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording0_90_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording0_90_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording0_90_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording0_90_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording1_90_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording1_90_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording1_90_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording1_90_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A1_1_45/recording0_135_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_do

['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording0_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording0_180_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording0_180_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording0_180_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording1_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording1_180_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording1_180_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording1_180_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/A2_1_90/recording0_225_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/

['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording0_315_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording0_315_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording0_315_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording0_315_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording1_315_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording1_315_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording1_315_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B0_3_0/recording1_315_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B1_3_45/recording0_0_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_down

['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording0_45_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording0_45_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording0_45_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording0_45_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording1_45_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording1_45_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording1_45_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording1_45_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_downstairs_nowall_trial1/B2_3_90/recording0_90_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s1/s1_dow

...

['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording0_135_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording0_135_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording0_135_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording0_135_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording1_135_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording1_135_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording1_135_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording1_135_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/recording0_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/A2_1_90/r

['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording0_315_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording0_315_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording0_315_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording0_315_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording1_315_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording1_315_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording1_315_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B0_3_0/recording1_315_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B1_3_45/recording0_0_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B1_3_45/recording0_

['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording0_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording0_180_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording0_180_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording0_180_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording1_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording1_180_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording1_180_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording1_180_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/recording0_225_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/B2_3_90/r

['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording0_0_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording0_0_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording0_0_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording0_0_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording1_0_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording1_0_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording1_0_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording1_0_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording0_45_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C1_5_45/recording0_45_2.wa

['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording0_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording0_180_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording0_180_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording0_180_4.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording1_180_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording1_180_2.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording1_180_3.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording1_180_4.wav']
['/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/recording0_225_1.wav', '/home/em/school/cpsc554x/554xproj/data/raw/s9/s9_upstairs_wall_trial2/C2_5_90/r

In [9]:
train_df

,sample,trial,angle_pos,dov,gccphat_0_1_maxshift,gccphat_0_1_auc,gccphat_0_1_peakval,gccphat_0_2_maxshift,gccphat_0_2_auc,gccphat_0_2_peakval,...,gccphatval_2_3_13,gccphatval_2_3_14,gccphatval_2_3_15,gccphatval_2_3_16,gccphatval_2_3_17,gccphatval_2_3_18,gccphatval_2_3_19,gccphatval_2_3_20,gccphatval_2_3_21,gccphatval_2_3_22
0,s1,downstairs_nowall_trial1,A0_1_0,0,0.000000,0.929901,0.918024,-0.000125,0.802662,-0.081439,...,0.000874,0.057902,0.000011,0.001541,-0.052348,0.000403,0.000849,0.023651,0.000048,0.001163
1,s1,downstairs_nowall_trial1,A0_1_0,45,0.000000,0.935565,0.861144,-0.000125,0.829486,-0.119773,...,0.000171,-0.050401,0.000652,0.000966,0.017136,0.000918,-0.000259,-0.035492,0.000443,0.000769
2,s1,downstairs_nowall_trial1,A0_1_0,90,0.000000,0.977008,0.580438,-0.000063,0.830345,0.062272,...,-0.000743,-0.007790,-0.000195,0.001031,0.055236,0.002341,-0.001265,-0.038105,-0.000551,0.000891
3,s1,downstairs_nowall_trial1,A0_1_0,135,0.000000,0.909018,0.372427,-0.000063,0.872800,0.160522,...,-0.001413,0.076345,-0.000457,0.001607,0.070554,0.003886,-0.000515,-0.046689,-0.001151,0.001109
4,s1,downstairs_nowall_trial1,A0_1_0,180,0.000000,0.897188,0.428891,-0.000063,0.823948,0.131519,...,-0.000334,0.157349,0.002033,0.001508,0.063899,-0.000289,0.000181,-0.036023,0.000971,0.000908
5,s1,downstairs_nowall_trial1,A0_1_0,225,0.000000,0.912079,0.394519,-0.000125,0.853461,0.145651,...,-0.000639,0.207072,-0.000717,0.000789,0.081844,0.002312,-0.000746,-0.015713,-0.000417,0.000286
6,s1,downstairs_nowall_trial1,A0_1_0,270,0.000000,0.945834,0.703730,-0.000125,0.820558,0.046309,...,0.001192,0.165007,0.001008,0.001740,0.006061,0.002051,0.000718,0.045567,0.000334,0.001064
7,s1,downstairs_nowall_trial1,A0_1_0,315,0.000000,0.926344,0.848013,-0.000125,0.834676,-0.007897,...,0.001120,0.154116,0.001274,0.001122,-0.091895,0.000595,0.000989,0.052921,0.000680,0.000992
8,s1,downstairs_nowall_trial1,A1_1_45,0,0.000063,0.947125,-0.123378,0.000000,0.855856,0.898838,...,0.001165,0.097836,0.001311,0.000044,-0.068905,0.000683,0.000548,0.008709,0.001353,-0.000853
9,s1,downstairs_nowall_trial1,A1_1_45,45,0.000063,0.954734,-0.111605,0.000000,0.859415,0.779354,...,0.000457,0.141664,0.001670,-0.000498,-0.076078,0.001355,0.000187,0.029534,0.001884,-0.000281


In [10]:
test_df

,sample,trial,angle_pos,dov,gccphat_0_1_maxshift,gccphat_0_1_auc,gccphat_0_1_peakval,gccphat_0_2_maxshift,gccphat_0_2_auc,gccphat_0_2_peakval,...,gccphatval_2_3_13,gccphatval_2_3_14,gccphatval_2_3_15,gccphatval_2_3_16,gccphatval_2_3_17,gccphatval_2_3_18,gccphatval_2_3_19,gccphatval_2_3_20,gccphatval_2_3_21,gccphatval_2_3_22
0,s2,upstairs_nowall_trial1,A0_1_0,0,0.000000,0.906094,0.777668,-0.000125,0.777412,-0.092405,...,-0.001048,-0.053518,-0.000374,0.003315,0.055637,0.004541,-0.001846,-0.027237,-0.001704,0.002633
1,s2,upstairs_nowall_trial1,A0_1_0,45,0.000000,0.889957,0.723852,-0.000063,0.790713,-0.086907,...,0.001355,-0.096628,0.000817,0.002954,0.101104,0.002984,0.000775,-0.086892,-0.000365,0.002554
2,s2,upstairs_nowall_trial1,A0_1_0,90,0.000000,0.923695,0.475861,-0.000063,0.771754,0.037184,...,-0.001145,-0.019927,0.000924,0.002598,0.115176,0.003542,-0.000553,-0.045375,0.000102,0.002808
3,s2,upstairs_nowall_trial1,A0_1_0,135,0.000000,0.877843,0.282748,-0.000063,0.762944,0.134039,...,0.001223,0.138240,0.000327,0.002508,0.063784,0.004482,-0.000374,-0.009642,0.000209,0.002245
4,s2,upstairs_nowall_trial1,A0_1_0,180,0.000000,0.885411,0.320892,-0.000125,0.863471,0.153375,...,0.001511,0.262882,0.004435,0.000473,0.025710,0.001319,0.000560,0.002125,0.000755,-0.000575
5,s2,upstairs_nowall_trial1,A0_1_0,225,-0.000063,0.930358,0.228290,-0.000125,0.889631,0.095525,...,0.003212,0.309301,0.001986,0.002558,0.162648,0.003518,0.000304,-0.021663,-0.000150,0.003040
6,s2,upstairs_nowall_trial1,A0_1_0,270,0.000000,0.882788,0.333901,-0.000125,0.863589,0.116334,...,0.001592,0.267739,0.002285,0.001201,0.113669,0.000133,-0.000046,-0.002352,0.001906,0.001375
7,s2,upstairs_nowall_trial1,A0_1_0,315,0.000000,0.904434,0.675269,-0.000125,0.801284,0.008973,...,0.000551,0.099438,0.001400,0.002356,-0.009452,0.001778,0.001459,0.013973,0.000618,0.002274
8,s2,upstairs_nowall_trial1,A1_1_45,0,0.000063,0.915568,-0.097916,0.000000,0.854675,0.770256,...,0.002215,0.165482,0.002662,-0.000504,-0.048790,0.000099,0.002702,0.002310,0.002177,-0.001633
9,s2,upstairs_nowall_trial1,A1_1_45,45,0.000063,0.885851,-0.114436,0.000000,0.841895,0.704664,...,0.002939,0.179595,0.002026,0.001128,-0.069873,0.000514,0.003180,0.038057,0.002392,-0.000061


In [11]:
train_df.to_csv('data/featurized/featurized_train_data.csv', index=False)
test_df.to_csv('data/featurized/featurized_test_data.csv', index=False)